#Задача учебная

Разбейте корпус текстов (или их часть) на темы

## Setup

In [ ]:
#pip install pyLDAvis --upgrade
!pip install pyLDAvis

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from gensim import models
from gensim.corpora import Dictionary, MmCorpus
import nltk
import pandas as pd

from google.colab import drive

import string
import re
from functools import reduce

In [ ]:
!pip install corus
from corus import load_ne5

     |████████████████████████████████| 83 kB 1.4 MB/s 


In [ ]:
!pip install pymorphy2
import pymorphy2

     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 8.2 MB 10.9 MB/s 


## Load Data

### Попробуем с русскими новостями

In [ ]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2021-09-28 16:24:54--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 80.240.100.4
Connecting to www.labinform.ru (www.labinform.ru)|80.240.100.4|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M   421KB/s    in 4.4s    

2021-09-28 16:24:59 (421 KB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [ ]:
!unzip collection5.zip 
!rm collection5.zip

In [ ]:
from corus import load_ne5
path = '/content/Collection5'
records = load_ne5(path)
print(next(records))
print(len(list(records))+1)

Ne5Markup(id='355', text='Командование турецкой армии подало в отставку\r\n\r\nНачальник Генштаба Турции Ышик Кошанер, а также главкомы Сухопутных войск, ВВС и ВМС в пятницу, 29 июля, подали в отставку и покинули свои посты, передает Reuters. О причинах добровольной отставки не сообщается.\r\n\r\nОднако не для кого не секрет, что в последнее время глава Генштаба Турции находился в жесткой оппозиции к действующему премьер-министру страны Тайипу Эрдогану. Отметим, что на следующей неделе должно было состояться масштабное заседание Высшего совета Вооруженных сил, в котором, как ожидалось, должен был принять участие и глава правительства.\r\n\r\nВ ходе заседания совета должен был быть решен вопрос о повышении в звании ряда офицеров вооруженных сил. Однако ранее ряд министров требовали отправить в отставку 17 высокопоставленных военнослужащих, подозреваемых в участии в антиправительственном заговоре в 2003г. Отметим, что реформы премьер-министра Т.Эрдогана снизили влияние армии на руководст

In [ ]:
%%time
records = load_ne5(path)
sequences_ = []
for i,record in enumerate(records):
  text = record.text.lower()
  text = re.sub('\r\n',' ',text)
  #text = re.sub('[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]+', " ", text)
  text = re.sub('[' + string.punctuation + ']+', " ", text)
  text = text.strip()
  if i<2: 
    print(text)
  sequences_.append(text)

командование турецкой армии подало в отставку  начальник генштаба турции ышик кошанер  а также главкомы сухопутных войск  ввс и вмс в пятницу  29 июля  подали в отставку и покинули свои посты  передает reuters  о причинах добровольной отставки не сообщается   однако не для кого не секрет  что в последнее время глава генштаба турции находился в жесткой оппозиции к действующему премьер министру страны тайипу эрдогану  отметим  что на следующей неделе должно было состояться масштабное заседание высшего совета вооруженных сил  в котором  как ожидалось  должен был принять участие и глава правительства   в ходе заседания совета должен был быть решен вопрос о повышении в звании ряда офицеров вооруженных сил  однако ранее ряд министров требовали отправить в отставку 17 высокопоставленных военнослужащих  подозреваемых в участии в антиправительственном заговоре в 2003г  отметим  что реформы премьер министра т эрдогана снизили влияние армии на руководство страны и существенно ослабили ее позиции 

In [ ]:
sequences_[0]

'командование турецкой армии подало в отставку  начальник генштаба турции ышик кошанер  а также главкомы сухопутных войск  ввс и вмс в пятницу  29 июля  подали в отставку и покинули свои посты  передает reuters  о причинах добровольной отставки не сообщается   однако не для кого не секрет  что в последнее время глава генштаба турции находился в жесткой оппозиции к действующему премьер министру страны тайипу эрдогану  отметим  что на следующей неделе должно было состояться масштабное заседание высшего совета вооруженных сил  в котором  как ожидалось  должен был принять участие и глава правительства   в ходе заседания совета должен был быть решен вопрос о повышении в звании ряда офицеров вооруженных сил  однако ранее ряд министров требовали отправить в отставку 17 высокопоставленных военнослужащих  подозреваемых в участии в антиправительственном заговоре в 2003г  отметим  что реформы премьер министра т эрдогана снизили влияние армии на руководство страны и существенно ослабили ее позиции

### Remove common words 
с помощью списка стоп-слов

смотрим [сюда](https://radimrehurek.com/gensim/auto_examples/core/run_similarity_queries.html#creating-the-corpus) и [сюда](https://habr.com/ru/post/417167/)

In [ ]:
stoplist = ["c","а","алло","без","белый","близко","более","больше","большой","будем","будет","будете","будешь","будто","буду","будут","будь","бы","бывает","бывь","был","была","были","было","быть","в","важная","важное","важные","важный","вам","вами","вас","ваш","ваша","ваше","ваши","вверх","вдали","вдруг","ведь","везде","вернуться","весь","вечер","взгляд","взять","вид","видел","видеть","вместе","вне","вниз","внизу","во","вода","война","вокруг","вон","вообще","вопрос","восемнадцатый","восемнадцать","восемь","восьмой","вот","впрочем","времени","время","все","все еще","всегда","всего","всем","всеми","всему","всех","всею","всю","всюду","вся","всё","второй","вы","выйти","г","где","главный","глаз","говорил","говорит","говорить","год","года","году","голова","голос","город","да","давать","давно","даже","далекий","далеко","дальше","даром","дать","два","двадцатый","двадцать","две","двенадцатый","двенадцать","дверь","двух","девятнадцатый","девятнадцать","девятый","девять","действительно","дел","делал","делать","делаю","дело","день","деньги","десятый","десять","для","до","довольно","долго","должен","должно","должный","дом","дорога","друг","другая","другие","других","друго","другое","другой","думать","душа","е","его","ее","ей","ему","если","есть","еще","ещё","ею","её","ж","ждать","же","жена","женщина","жизнь","жить","за","занят","занята","занято","заняты","затем","зато","зачем","здесь","земля","знать","значит","значить","и","иди","идти","из","или","им","имеет","имел","именно","иметь","ими","имя","иногда","их","к","каждая","каждое","каждые","каждый","кажется","казаться","как","какая","какой","кем","книга","когда","кого","ком","комната","кому","конец","конечно","которая","которого","которой","которые","который","которых","кроме","кругом","кто","куда","лежать","лет","ли","лицо","лишь","лучше","любить","люди","м","маленький","мало","мать","машина","между","меля","менее","меньше","меня","место","миллионов","мимо","минута","мир","мира","мне","много","многочисленная","многочисленное","многочисленные","многочисленный","мной","мною","мог","могу","могут","мож","может","может быть","можно","можхо","мои","мой","мор","москва","мочь","моя","моё","мы","на","наверху","над","надо","назад","наиболее","найти","наконец","нам","нами","народ","нас","начала","начать","наш","наша","наше","наши","не","него","недавно","недалеко","нее","ней","некоторый","нельзя","нем","немного","нему","непрерывно","нередко","несколько","нет","нею","неё","ни","нибудь","ниже","низко","никакой","никогда","никто","никуда","ним","ними","них","ничего","ничто","но","новый","нога","ночь","ну","нужно","нужный","нх","о","об","оба","обычно","один","одиннадцатый","одиннадцать","однажды","однако","одного","одной","оказаться","окно","около","он","она","они","оно","опять","особенно","остаться","от","ответить","отец","откуда","отовсюду","отсюда","очень","первый","перед","писать","плечо","по","под","подойди","подумать","пожалуйста","позже","пойти","пока","пол","получить","помнить","понимать","понять","пор","пора","после","последний","посмотреть","посреди","потом","потому","почему","почти","правда","прекрасно","при","про","просто","против","процентов","путь","пятнадцатый","пятнадцать","пятый","пять","работа","работать","раз","разве","рано","раньше","ребенок","решить","россия","рука","русский","ряд","рядом","с","с кем","сам","сама","сами","самим","самими","самих","само","самого","самой","самом","самому","саму","самый","свет","свое","своего","своей","свои","своих","свой","свою","сделать","сеаой","себе","себя","сегодня","седьмой","сейчас","семнадцатый","семнадцать","семь","сидеть","сила","сих","сказал","сказала","сказать","сколько","слишком","слово","случай","смотреть","сначала","снова","со","собой","собою","советский","совсем","спасибо","спросить","сразу","стал","старый","стать","стол","сторона","стоять","страна","суть","считать","т","та","так","такая","также","таки","такие","такое","такой","там","твои","твой","твоя","твоё","те","тебе","тебя","тем","теми","теперь","тех","то","тобой","тобою","товарищ","тогда","того","тоже","только","том","тому","тот","тою","третий","три","тринадцатый","тринадцать","ту","туда","тут","ты","тысяч","у","увидеть","уж","уже","улица","уметь","утро","хороший","хорошо","хотел бы","хотеть","хоть","хотя","хочешь","час","часто","часть","чаще","чего","человек","чем","чему","через","четвертый","четыре","четырнадцатый","четырнадцать","что","чтоб","чтобы","чуть","шестнадцатый","шестнадцать","шестой","шесть","эта","эти","этим","этими","этих","это","этого","этой","этом","этому","этот","эту","я","являюсь"]
# https://github.com/stopwords-iso/stopwords-ru/blob/master/stopwords-ru.json


In [ ]:
%%time
sequences = [item.split() for item in sequences_]
print(sequences[0])

['командование', 'турецкой', 'армии', 'подало', 'в', 'отставку', 'начальник', 'генштаба', 'турции', 'ышик', 'кошанер', 'а', 'также', 'главкомы', 'сухопутных', 'войск', 'ввс', 'и', 'вмс', 'в', 'пятницу', '29', 'июля', 'подали', 'в', 'отставку', 'и', 'покинули', 'свои', 'посты', 'передает', 'reuters', 'о', 'причинах', 'добровольной', 'отставки', 'не', 'сообщается', 'однако', 'не', 'для', 'кого', 'не', 'секрет', 'что', 'в', 'последнее', 'время', 'глава', 'генштаба', 'турции', 'находился', 'в', 'жесткой', 'оппозиции', 'к', 'действующему', 'премьер', 'министру', 'страны', 'тайипу', 'эрдогану', 'отметим', 'что', 'на', 'следующей', 'неделе', 'должно', 'было', 'состояться', 'масштабное', 'заседание', 'высшего', 'совета', 'вооруженных', 'сил', 'в', 'котором', 'как', 'ожидалось', 'должен', 'был', 'принять', 'участие', 'и', 'глава', 'правительства', 'в', 'ходе', 'заседания', 'совета', 'должен', 'был', 'быть', 'решен', 'вопрос', 'о', 'повышении', 'в', 'звании', 'ряда', 'офицеров', 'вооруженных', '

In [ ]:
%%time
sequences_clean= [
    [word for word in document.lower().split() if word not in stoplist]
    for document in sequences_
]
print(sequences_clean[0])

['командование', 'турецкой', 'армии', 'подало', 'отставку', 'начальник', 'генштаба', 'турции', 'ышик', 'кошанер', 'главкомы', 'сухопутных', 'войск', 'ввс', 'вмс', 'пятницу', '29', 'июля', 'подали', 'отставку', 'покинули', 'посты', 'передает', 'reuters', 'причинах', 'добровольной', 'отставки', 'сообщается', 'секрет', 'последнее', 'глава', 'генштаба', 'турции', 'находился', 'жесткой', 'оппозиции', 'действующему', 'премьер', 'министру', 'страны', 'тайипу', 'эрдогану', 'отметим', 'следующей', 'неделе', 'состояться', 'масштабное', 'заседание', 'высшего', 'совета', 'вооруженных', 'сил', 'котором', 'ожидалось', 'принять', 'участие', 'глава', 'правительства', 'ходе', 'заседания', 'совета', 'решен', 'повышении', 'звании', 'ряда', 'офицеров', 'вооруженных', 'сил', 'ранее', 'министров', 'требовали', 'отправить', 'отставку', '17', 'высокопоставленных', 'военнослужащих', 'подозреваемых', 'участии', 'антиправительственном', 'заговоре', '2003г', 'отметим', 'реформы', 'премьер', 'министра', 'эрдогана'

In [ ]:
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel

In [ ]:
import warnings
warnings.filterwarnings("ignore") 

In [ ]:
%%time
common_dictionary = Dictionary(sequences_clean) #подаем список списков слов, формируем слова
common_corpus = [common_dictionary.doc2bow(text) for text in sequences_clean] #превращаем предложение в вектор bow

lda = LdaModel(common_corpus, num_topics=10) #тренируем модель

CPU times: user 2.98 s, sys: 45.1 ms, total: 3.02 s
Wall time: 3.03 s


In [ ]:
#import pyLDAvis.gensim as gensimvis
warnings.filterwarnings("ignore") 
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_lda.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [ ]:
%%time
vis_data = gensimvis.prepare(lda, common_corpus, common_dictionary) #подготовим объет визуальных данных по пространству топиков

CPU times: user 1.4 s, sys: 89 ms, total: 1.49 s
Wall time: 4.85 s


In [ ]:
lda.show_topics()

[(0,
  '0.004*"410" + 0.004*"220" + 0.004*"219" + 0.004*"319" + 0.003*"609" + 0.003*"1135" + 0.003*"162" + 0.003*"2074" + 0.003*"186" + 0.003*"719"'),
 (1,
  '0.003*"1308" + 0.003*"101" + 0.003*"89" + 0.003*"609" + 0.002*"220" + 0.002*"319" + 0.002*"378" + 0.002*"26" + 0.002*"751" + 0.002*"410"'),
 (2,
  '0.006*"323" + 0.004*"904" + 0.003*"410" + 0.003*"609" + 0.003*"117" + 0.003*"319" + 0.003*"48" + 0.003*"71" + 0.003*"162" + 0.003*"219"'),
 (3,
  '0.006*"609" + 0.005*"319" + 0.004*"323" + 0.003*"219" + 0.003*"2074" + 0.003*"26" + 0.003*"410" + 0.003*"225" + 0.002*"162" + 0.002*"286"'),
 (4,
  '0.004*"220" + 0.004*"323" + 0.003*"410" + 0.003*"162" + 0.003*"219" + 0.003*"319" + 0.003*"26" + 0.003*"101" + 0.002*"90" + 0.002*"225"'),
 (5,
  '0.005*"162" + 0.005*"89" + 0.004*"220" + 0.004*"323" + 0.004*"410" + 0.004*"219" + 0.003*"319" + 0.003*"71" + 0.003*"993" + 0.003*"26"'),
 (6,
  '0.006*"323" + 0.006*"410" + 0.004*"319" + 0.004*"48" + 0.004*"162" + 0.004*"219" + 0.003*"1135" + 0.003*

In [ ]:
topic_id = 1
terms = lda.get_topic_terms(topic_id)
print(terms)
for t in terms:
 print(common_dictionary[t[0]])

[(1308, 0.0033622698), (101, 0.003213674), (89, 0.0028707462), (609, 0.0025537948), (220, 0.0024982446), (319, 0.0023644322), (378, 0.0021916402), (26, 0.0021911028), (751, 0.0020972078), (410, 0.0020204782)]
москвы
ранее
правительства
области
президента
россии
заявил
глава
сша
пост


In [ ]:
pyLDAvis.display(vis_data) #визуализируем данные

## Разбейте найденный Вами для задания LIGHT корпус текстов (или их часть) на темы.

### Готовим модель схожести (подобия) по исходному корпусу текстов

In [ ]:
# смотреим сюда
# https://radimrehurek.com/gensim/auto_examples/core/run_similarity_queries.html#sphx-glr-auto-examples-core-run-similarity-queries-py
from gensim import models
lsi = models.LsiModel(common_corpus, id2word=common_dictionary, num_topics=4)

In [ ]:
import warnings
warnings.filterwarnings("ignore") 

In [ ]:
%%time
# Initializing query structures
from gensim import similarities
index = similarities.MatrixSimilarity(lsi[common_corpus])  # transform corpus to LSI space and index it

CPU times: user 144 ms, sys: 3.98 ms, total: 148 ms
Wall time: 149 ms


In [ ]:
index.save('/tmp/deerwester.index')
index = similarities.MatrixSimilarity.load('/tmp/deerwester.index')

In [ ]:
%%time
# готовим модель для выделения похожих слов (отдельно)
import gensim
w2v = gensim.models.Word2Vec(sequences_clean, min_count=1)
print(w2v.most_similar(positive=['президент'], topn=5))

[('назначил', 0.9996678829193115), ('указом', 0.9994470477104187), ('путин', 0.9993948340415955), ('своим', 0.9993106126785278), ('д', 0.9992934465408325)]
CPU times: user 9.69 s, sys: 147 ms, total: 9.84 s
Wall time: 8.36 s


### Выбираем темы

In [ ]:
topics = [
  'назначил уволил освободил наказал',
  #'спорт соревнование награда медаль', # не нашлось новостей по этой теме
  #'шпионаж разведка следствие',
  'политика кризис отставка',
  #'наркотики',
  'коррупция взятки'
]

### Токенизируем и превращаем в векторы BOW и LSI 
(Latent semantic indexing - скрытая семантическаяя индексация)

In [ ]:
topics_tokenized = [topic.lower().split() for topic in topics]
vecs_bow = [ common_dictionary.doc2bow(topic_tokenized) for topic_tokenized in topics_tokenized]
vecs_lsi = [lsi[vec_bow] for vec_bow in vecs_bow]             # convert the query to LSI space
for i, topic in enumerate(topics):
  print(topics[i])
  print(topics_tokenized[i])
  print(vecs_bow[i])
  print(vecs_lsi[i]) 

назначил уволил освободил наказал
['назначил', 'уволил', 'освободил', 'наказал']
[(299, 1), (1275, 1), (2347, 1), (9910, 1)]
[(0, 0.10085063373108334), (1, -0.07391668182784518), (2, 0.00515882616684869), (3, 0.021649009251222814)]
политика кризис отставка
['политика', 'кризис', 'отставка']
[(525, 1), (1122, 1), (3228, 1)]
[(0, 0.029236830580338026), (1, 0.029613784354439083), (2, -0.0028458259967759332), (3, 0.03709018629356321)]
коррупция взятки
['коррупция', 'взятки']
[(1048, 1), (30489, 1)]
[(0, 0.0036628753212530818), (1, -0.0009104910164418004), (2, -0.0008658206386228827), (3, -0.0045164538599080754)]


In [ ]:
# Определяем  схожесть кадого документа из корпуса с заданными темами
# Performing queries
# To obtain similarities of our query document against the nine indexed documents:
lst_sims = [index[vec_lsi] for vec_lsi in vecs_lsi]  # perform a similarity query against the corpus
print(len(lst_sims[0]), list(enumerate(lst_sims[0])))  # print (document_number, document_similarity) 2-tuple

1000 [(0, 0.39845034), (1, 0.4247514), (2, 0.83734375), (3, 0.22086954), (4, 0.3048641), (5, 0.52865636), (6, 0.27764678), (7, 0.20700864), (8, 0.51172197), (9, 0.18438628), (10, 0.90194076), (11, 0.27814275), (12, 0.9858359), (13, 0.9432178), (14, 0.41456762), (15, 0.9664937), (16, 0.72694916), (17, 0.4626019), (18, 0.3871988), (19, 0.80734307), (20, 0.19568029), (21, 0.75490916), (22, 0.21885641), (23, 0.33322525), (24, 0.40136296), (25, 0.991893), (26, 0.3302003), (27, 0.5298091), (28, 0.25261453), (29, 0.79191077), (30, 0.37340477), (31, 0.31443846), (32, 0.61313057), (33, 0.38515574), (34, 0.92481107), (35, 0.52243257), (36, 0.9057221), (37, 0.31955934), (38, 0.33317304), (39, 0.61101925), (40, 0.96797043), (41, 0.6682778), (42, 0.79956686), (43, 0.73732954), (44, 0.80911463), (45, 0.286062), (46, 0.3711688), (47, 0.4503256), (48, 0.54773176), (49, 0.053698167), (50, 0.31172085), (51, 0.5039973), (52, 0.38462073), (53, 0.49016818), (54, 0.38902962), (55, 0.46423367), (56, 0.342710

In [ ]:
%%time
# Сортируем по возрастанию схожести
# Cosine measure returns similarities in the range <-1, 1> (the greater, the more similar), so that the first document has a score of 0.99809301 etc.
# With some standard Python magic we sort these similarities into descending order, and obtain the final answer to the query *new_topic_text:
lst_sims_sorted = [sorted(enumerate(sims), key=lambda item: -item[1]) for sims in lst_sims]

CPU times: user 2.07 ms, sys: 1.01 ms, total: 3.08 ms
Wall time: 3.23 ms


### Формируем список похожих слов - дополнительно (для интереса)


In [ ]:
num_sim_words = 5
topn = num_sim_words
# Выбираем похожие слова для каждого токена (слова) темы
sim_words_tuples = [[[w2v.most_similar(positive=[token], topn=num_sim_words)]  for token in topic] for topic in topics_tokenized ]

# Оставляем только слова, убирая веса схожести
sim_words = [ [tuple_[0]    for token_tuples in tokens_tuples for tuples in token_tuples for tuple_ in tuples] for tokens_tuples in sim_words_tuples ] 

# Убираем дублирование
sim_words = [list(set(sim_words_w))  for sim_words_w in sim_words] 

for i, topic in enumerate(topics_tokenized):
  print("topic", topic)
  print("similar words", len(sim_words[i]), sim_words[i])

topic ['назначил', 'уволил', 'освободил', 'наказал']
similar words 16 ['своим', 'уволил', 'д', 'указом', 'освободил', 'дальнейшего', 'летели', 'каган', 'подписал', 'россии', 'назначил', 'президент', 'отечественных', 'генералов', 'подытожил', 'указ']
topic ['политика', 'кризис', 'отставка']
similar words 15 ['словам', 'кандидатов', 'выборы', 'поста', 'известно', '2004', 'является', 'ранее', 'своем', 'сша', 'директор', 'данным', 'бизнес', 'работу', 'считает']
topic ['коррупция', 'взятки']
similar words 10 ['20', 'амирову', 'осуществить', 'гранатами', 'директором', 'работе', 'орлы', 'возглавлял', 'почета', 'информации']


Своеобразный, конечно, набор текстов

### Выводим по 5 текстов для каждой темы

In [ ]:
green_text = 32
pink_text = 35
green_higlight = 42
pink_higlight = 45
gray_higlight = 47
def colorText(color_, txt):
    return(f"\033[2;{str(color_)}m" + txt +"\033[0;0m")
    #return("\033[2;{num}m" +txt {num} \033[0;0m".format(num=str(color_)))
print(colorText(pink_text, 'Так будут выглядеть слова из темы'))    
print(colorText(gray_higlight, 'А так будут выглядеть похожие слова'))    

Так будут выглядеть слова из темы
А так будут выглядеть похожие слова


In [ ]:
def print_sims_texts(sims, topic_index, num):
  for i, (doc_position, doc_score) in enumerate(sims):
    if i>=num:
      break
    print(doc_position, doc_score, sequences_clean[doc_position])
    print("-->", end="")
    for tok in sequences_[doc_position].split():
      if tok.lower() in topics_tokenized[topic_index] :
        print(colorText(pink_text, tok), " ", end="")  
      elif tok.lower() in sim_words[topic_index]:
        print(colorText(gray_higlight, tok), " ", end="")  
      else:
        print(tok, " ", end="")
    print()
    

In [ ]:
for i, sims in enumerate(lst_sims_sorted):
  print(f"Тема: {topics[i]}, Похожие слова: {sim_words[i]}")
  print_sims_texts(sims, i, 5)
  print()

Тема: назначил уволил освободил наказал, Похожие слова: ['своим', 'уволил', 'д', 'указом', 'освободил', 'дальнейшего', 'летели', 'каган', 'подписал', 'россии', 'назначил', 'президент', 'отечественных', 'генералов', 'подытожил', 'указ']
728 0.9976117 ['д', 'медведев', 'назначил', 'начальника', 'гувд', 'ставрополья', 'главу', 'мвд', 'дагестана', 'президент', 'россии', 'дмитрий', 'медведев', 'подписал', 'указ', 'присвоении', 'специальных', 'званий', 'высшего', 'начальствующего', 'состава', 'сотрудникам', 'органов', 'внутренних', 'российской', 'федерации', 'назначении', 'должность', 'сотрудников', 'органов', 'внутренних', 'российской', 'федерации', 'сообщает', 'пресс', 'служба', 'главы', 'государства', 'александр', 'горовой', 'назначен', 'начальником', 'главного', 'управления', 'министерства', 'внутренних', 'рф', 'ставропольскому', 'краю', 'соответствии', 'указом', 'присвоено', 'звание', 'генерал', 'лейтенанта', 'полиции', 'указом', 'абдурашид', 'магомедов', 'назначен', 'пост', 'министра',

Скрытая сегментация (LSI) дает результаты и без похожих слов (или, возможно, похожих слов маловато) 

## Посмотрите, к каким темам принадлежат выбранные Вами тексты. Относятся ли они к одной и той же теме, “выученной” LDA?

In [ ]:
print(lda.get_document_topics(common_dictionary.doc2bow(sequences_clean[801])))
print(lda.get_topic_terms(8))
print([common_dictionary[t[0]] for t in lda.get_topic_terms(8)])

[(6, 0.99724716)]
[(268, 0.005496549), (323, 0.004751271), (319, 0.0042392793), (410, 0.0032050714), (1458, 0.0028551603), (220, 0.0021615678), (751, 0.0021393264), (26, 0.0021259177), (219, 0.0020656101), (609, 0.0019280344)]
['—', 'рф', 'россии', 'пост', 'мвд', 'президента', 'сша', 'глава', 'президент', 'области']


In [ ]:
def print_sims_texts_compare_topics_LDA(sims, topic_index, num_texts_out):
  lda_topics_all = []
  for i, (doc_position, doc_score) in enumerate(sims):
    if i>=num_texts_out:
      break
    print(doc_position, doc_score, sequences_clean[doc_position])
    print("--> ", end="")
    for tok in sequences_[doc_position].split():
      if tok.lower() in topics_tokenized[topic_index] :
        print(colorText(pink_text, tok), " ", end="")  
      elif tok.lower() in sim_words[topic_index]:
        print(colorText(gray_higlight, tok), " ", end="")  
      else:
        print(tok, " ", end="")
    print()
    lda_topics = lda.get_document_topics(common_dictionary.doc2bow(sequences_clean[int(doc_position)]))
    print(f'--> lda_topics: {lda_topics}')
    lda_topics_all.append(lda_topics)
    for lda_topic in lda_topics:
      terms = lda.get_topic_terms(lda_topic[0])
      print(f'--> lda_terms[{lda_topic[0]}] -->', [common_dictionary[t[0]] for t in terms])
    print()
  print('lda_topics_all', lda_topics_all)
  
  # список топиков  в разрезе каждого документа из "нашей" темы
  lda_topics_all_short = [[t[0] for t in lda_topic_ ] for lda_topic_ in lda_topics_all ]
  print('lda_topics_all', lda_topics_all_short)
  
  # сплошной список топиков  для "нашей" темы
  lda_topics_all_flat = list(set([topic_num for lda_topic_short in lda_topics_all_short for topic_num in lda_topic_short]))
  print(lda_topics_all_flat)

  # считаем сколько раз тексты "нашей" темы оказалась в списе топиков LDA
  lda_topics_all_cnt2 = [     [ 1 if num in lda_topic_short  else 0  for num in lda_topics_all_flat                                      
                              ]
                          for lda_topic_short in lda_topics_all_short  ]
  print(lda_topics_all_cnt2)
  lda_topics_all_cnt3 = [reduce(lambda x,y: list(np.array(x) + np.array(y)), lda_topics_all_cnt2)] 
  print(lda_topics_all_cnt3)
  if np.array(lda_topics_all_cnt3).any() >= num_texts_out: 
    print(colorText(pink_text, "Есть совпадение хотя бы с одной 'выученной' темой LDA у всех текстов по 'нашей' теме")) 
  else:
    print(colorText(gray_higlight, "Нет совпадения у всех текстов по 'нашей' теме хотя бы с одной 'выученной' темой LDA"  )) 
  #  print()

In [ ]:
from functools import reduce
for i, sims in enumerate(lst_sims_sorted):
  print(f"Тема: {topics[i]}, Похожие слова: {sim_words[i]}")
  print_sims_texts_compare_topics_LDA(sims, i, 5)
  print()

Тема: назначил уволил освободил наказал, Похожие слова: ['своим', 'уволил', 'д', 'указом', 'освободил', 'дальнейшего', 'летели', 'каган', 'подписал', 'россии', 'назначил', 'президент', 'отечественных', 'генералов', 'подытожил', 'указ']
728 0.9976117 ['д', 'медведев', 'назначил', 'начальника', 'гувд', 'ставрополья', 'главу', 'мвд', 'дагестана', 'президент', 'россии', 'дмитрий', 'медведев', 'подписал', 'указ', 'присвоении', 'специальных', 'званий', 'высшего', 'начальствующего', 'состава', 'сотрудникам', 'органов', 'внутренних', 'российской', 'федерации', 'назначении', 'должность', 'сотрудников', 'органов', 'внутренних', 'российской', 'федерации', 'сообщает', 'пресс', 'служба', 'главы', 'государства', 'александр', 'горовой', 'назначен', 'начальником', 'главного', 'управления', 'министерства', 'внутренних', 'рф', 'ставропольскому', 'краю', 'соответствии', 'указом', 'присвоено', 'звание', 'генерал', 'лейтенанта', 'полиции', 'указом', 'абдурашид', 'магомедов', 'назначен', 'пост', 'министра',